# Face and expression detection

#### This application detects faces using the Haar Cascade Classifier method and classify facial expressions using the models trained on the FER-2013 dataset ('expression_model' file).

#### The application uses video capture from the webcam and outputs a summary of the captures facial expressions on closure.



### Import required libraries

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os
import cv2
import dlib
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.utils import img_to_array

### Load Haar Cascade Classifier

In [2]:
# Load the Haar Cascade Classifier method for face detection
# Reference: https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

### Data preprocessing

In [3]:
# Load the dataset
# Reference: https://www.kaggle.com/datasets/deadskull7/fer2013
df = pd.read_csv('fer2013.csv')

# Print unique values of the emotion col
df['emotion'].unique()

array([0, 2, 4, 6, 3, 5, 1], dtype=int64)

In [4]:
# Map expressions from dataset to string values
exp_map = {
    0: "Angry",
    1: "Disgust",
    2: "Fear",
    3: "Happy",
    4: "Sad",
    5: "Surprise",
    6: "Neutral"
}

# Create list with expression labels based on their integer values
exp_labels = [exp_map[i] for i in sorted(df['emotion'].unique())]

# Print expression labels
exp_labels

['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

## Expression detection using VGG16 model

#### Facial expression detection using model trained on FER2013 image dataset using VGG16

In [5]:
# Load the pre-trained expression recognition model
exp_model_VGG16 = load_model('expression_model_VGG16.h5')

In [7]:
# Frame count variable
frame_count = 0

# Initialize dictionary for observed expressions 
exp_obs = {expression: 0 for expression in exp_labels}

# Initialize video capture from webcam
video = cv2.VideoCapture(0)

while True:
    ret, image = video.read()
    if not ret:
        continue

    # Flip the image to match perspective from the FER-2013 database
    image = cv2.flip(image, 1)

    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Face detection using Haar Cascade Classifier
    face_detected = face_cascade.detectMultiScale(gray_image, scaleFactor=1.12, minNeighbors=6)

    # For each detected face (i.e. coordinates of detected face)
    for (x, y, w, h) in face_detected:
        # Draw rectangle around the detected face area: Used to identify the performace of the Haar Cascade Classifier
        #cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0))

        # Capture face region and repare image for model classification
        face_region = image[y:y + h, x:x + w]  # Capture face only (colour images for the custom VGG16 model)
        face_region = cv2.resize(face_region, (48, 48))  # Resize to 48x48
        face_region = img_to_array(face_region)  # Convert image to numpy array (pixels)
        face_region = np.expand_dims(face_region, axis=0) # Add batch dimension
        
        # Predict the emotion using the emotion recognition model
        exp_prediction = exp_model_VGG16.predict(face_region, verbose=0)  # Predict using the pre-trained model
        exp_max = np.argmax(exp_prediction)  # Index of expression with the highest probability
        exp_text = exp_labels[exp_max]  # Map expression to corresponding index

        # Update expression dictionary and frame count variable
        exp_obs[exp_text] += 1
        frame_count += 1

    # Display the predicted emotion top left corner
    cv2.putText(image, f"State: {exp_text}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (50, 255, 50), 2, cv2.LINE_AA)

    # Display image with expression classification
    cv2.imshow('Facial Expression Analysis', image)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the video capture object and close all OpenCV windows
video.release()
cv2.destroyAllWindows()

# Calculate percentages and determine the overall state
exp_percentages = {expression: (count / frame_count) * 100 for expression, count in exp_obs.items()}
overall_state = max(exp_obs, key=exp_obs.get)

# Print the expression state summary
print("\nExpression Summary:")
for expression, percentage in exp_percentages.items():
    print(f"{expression}: {percentage:.2f}%")

print(f"\nOverall State: {overall_state}")


Expression Summary:
Angry: 4.88%
Disgust: 0.00%
Fear: 14.33%
Happy: 21.34%
Sad: 18.60%
Surprise: 1.52%
Neutral: 39.33%

Overall State: Neutral


## Expression detection using custom CNN model

#### Facial expression detection using custom CNN model trained on FER2013 CSV format dataset 

In [8]:
# Load the pre-trained expression recognition model
exp_model_CSV = load_model('expression_model_CSV.h5')

In [10]:
# Frame count variable
frame_count = 0

# Initialize dictionary for observed expressions 
exp_obs = {expression: 0 for expression in exp_labels}

# Initialize video capture from webcam
video = cv2.VideoCapture(0)

while True:
    ret, image = video.read()
    if not ret:
        continue

    # Flip the image to match perspective from the FER-2013 database
    image = cv2.flip(image, 1)

    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Face detection using Haar Cascade Classifier
    face_detected = face_cascade.detectMultiScale(gray_image, scaleFactor=1.12, minNeighbors=6)

    # For each detected face (i.e. coordinates of detected face)
    for (x, y, w, h) in face_detected:
        # Draw rectangle around the detected face area: Used to identify the performace of the Haar Cascade Classifier
        #cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0))

        # Capture face region and repare image for model classification
        face_region = gray_image[y:y + h, x:x + w]  # Capture face only (greyscale images for the csv model)
        face_region = cv2.resize(face_region, (48, 48))  # Resize to 48x48
        face_region = img_to_array(face_region)  # Convert image to numpy array (pixels)
        face_region = np.expand_dims(face_region, axis=0) # Add batch dimension
        
        # Predict the emotion using the emotion recognition model
        exp_prediction = exp_model_CSV.predict(face_region, verbose=0)  # Predict using the pre-trained model
        exp_max = np.argmax(exp_prediction)  # Index of expression with the highest probability
        exp_text = exp_labels[exp_max]  # Map expression to corresponding index

        # Update expression dictionary and frame count variable
        exp_obs[exp_text] += 1
        frame_count += 1

    # Display the predicted emotion top left corner
    cv2.putText(image, f"State: {exp_text}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (50, 255, 50), 2, cv2.LINE_AA)

    # Display image with expression classification
    cv2.imshow('Facial Expression Analysis', image)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the video capture object and close all OpenCV windows
video.release()
cv2.destroyAllWindows()

# Calculate percentages and determine the overall state
exp_percentages = {expression: (count / frame_count) * 100 for expression, count in exp_obs.items()}
overall_state = max(exp_obs, key=exp_obs.get)

# Print the expression state summary
print("\nExpression Summary:")
for expression, percentage in exp_percentages.items():
    print(f"{expression}: {percentage:.2f}%")

print(f"\nOverall State: {overall_state}")


Expression Summary:
Angry: 3.82%
Disgust: 0.00%
Fear: 11.45%
Happy: 26.72%
Sad: 0.00%
Surprise: 13.74%
Neutral: 44.27%

Overall State: Neutral


## Conclusion